In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_5_new.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])

        for i in range(num):
            np.random.shuffle(train_data[i])
            train_data[i] = np.asarray(train_data[i])

        np.random.shuffle(train_data)   
        return train_data
    
    
    train_data = []
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
    
    np.random.shuffle(train_data)
    return np.asarray(train_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(train_data, split_type):
    
    print(split_type)
    
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)

        print(len(tx),len(tx[0]),len(tx[1]),len(tx[0][0]))
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(tx[test_i],ty[test_i])
            test_acc.append(classifier.train(
                X_train=train_x,
                y_train=train_y,
                X_eval=test_x,
                y_eval=test_y,
                categories=categories,
                epochs=50
            ))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            test_acc.append(classifier.train(
                X_train=tx[train_i],
                y_train=ty[train_i],
                X_eval=tx[test_i],
                y_eval=ty[test_i],
                categories=categories,
                epochs=50
            ))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)  # label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [5]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)
 
seq_length = 41  # seq_length = 37  TREC

Loading GloVe!
Completed!
400000 100 400000


In [6]:
class Classifier:

    def __init__(self, model, input_length, output_length):
        self.model = model
        self.input_length = input_length
        self.output_length = output_length

    def compile(self, batch_size=32):
        self._ds_x = tf.placeholder(tf.float32, [None, self.input_length])
        self._ds_y = tf.placeholder(tf.float32, [None, self.output_length])

        ds = tf.data.Dataset.from_tensor_slices((self._ds_x, self._ds_y))
        ds = ds.batch(batch_size)

        self._ds_it = ds.make_initializable_iterator()
        self._input, self._labels = self._ds_it.get_next()

        self._features = self.model(self._input)
        self._output = _create_dense_layer(self._features, self.output_length)

        self._create_acc_computations()
        self._create_backpropagation()

    def _create_acc_computations(self):
        self._predictions = tf.argmax(self._output, 1)
        labels = tf.argmax(self._labels, 1)
        self._accuracy = tf.reduce_mean(
            tf.cast(tf.equal(self._predictions, labels), 'float32'))

    def _create_backpropagation(self):
        losses = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self._output,
            labels=self._labels)
        self._loss = tf.reduce_mean(losses)

        optimizer = tf.train.AdamOptimizer(0.001)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        grads_and_vars = optimizer.compute_gradients(self._loss)

        self._train_op = optimizer.apply_gradients(
            grads_and_vars, global_step=global_step)

    def summary(self):
        print('input:', self._input.shape)
        self.model.summary()
        print('output:', self._output.shape)

    def train(self, X_train, y_train, X_eval, y_eval, categories, epochs=20, require_improve=3):
        
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())
        
        best_vac_acc = 0.0
        last_improved = 0
        
        for e in range(epochs):
            start_time = time.time()
            loss, acc = self._train(X_train, y_train, session)
            duration = time.time() - start_time

            val_loss, val_acc = self._eval(X_eval, y_eval, session)
            
            if val_acc > best_vac_acc:
                best_vac_acc = val_acc
                last_improved = e
                improved_str = '*'
            else:
                improved_str = ''
            
            output = 'Epoch: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
            print(output.format(e + 1, loss, acc, val_loss, val_acc, duration, improved_str))
            
            if e - last_improved > require_improve:
                print("No optimization for a long time, auto-stopping...")
                
                y_test_cls = np.argmax(y_eval, 1)  # 获得类别
                y_test_pred_cls = np.argmax(self.predict(X_eval, session), 1)
                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                
                # evaluate
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                print("Confusion Matrix...")
                print(metrics.confusion_matrix(y_test_cls, y_test_pred_cls))
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                break
        # endfor
        session.close()
        return accuracy_score

    def _train(self, X_train, y_train, session):
        import numpy as np

        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_train,
                self._ds_y: y_train
            })
        loss, acc, = [], []
        while True:
            try:
                _, vloss, vacc = session.run(
                    fetches=[self._train_op, self._loss, self._accuracy])

                loss.append(vloss)
                acc.append(vacc)
            except tf.errors.OutOfRangeError:
                break
        # endwhile

        loss, acc = np.mean(loss), np.mean(acc)
        return loss, acc

    def _eval(self, X_val, y_val, session):
        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_val,
                self._ds_y: y_val
            })

        loss, acc, = 0, 0
        while True:
            try:
                l, vloss, vacc = session.run(
                    fetches=[self._labels, self._loss, self._accuracy])

                loss += vloss * len(l)
                acc += vacc * len(l)
            except tf.errors.OutOfRangeError:
                break

        return loss / len(X_val), acc / len(X_val)

    def predict(self, X, session):
        session.run(self._ds_it.initializer,
                         feed_dict={
                             self._ds_x: X,
                             self._ds_y: np.empty((len(X), self.output_length))
                         }
                         )

        pred = list()
        while True:
            try:
                ppred = session.run(tf.nn.softmax(self._output))

                pred.extend(map(lambda l: l.tolist(), ppred))
            except tf.errors.OutOfRangeError:
                break

        return pred

def _create_dense_layer(x, output_length):
    '''Creates a dense layer
    '''
    input_size = x.shape[1].value
    W = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[input_size, output_length],
            stddev=0.1))
    b = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[output_length]))

    dense = tf.nn.xw_plus_b(x, W, b)

    return dense

In [7]:
class ShinContextualModel:
    '''
    Implementation proposal of: http://milab.snu.ac.kr/pub/BigComp2018.pdf
    '''
    def __init__(self, embedding, conv_size=3, num_layers=2):
        '''Constructor.
        # Parameters:
        embedding: numpy array representing the embedding.
        conv_size: Size of the convolutions. Number of words that takes each
            convolution step.
        num_layers: Number of recurrent convolutions.
        '''
        self._embedding = embedding
        self._conv_size = conv_size
        self._num_layers = num_layers

    def __call__(self, input):
        self._embedding_tf = self._create_embedding_layer(
            self._embedding, input)
        self._convolution_tf = self._create_convolutional_layers(
            self._conv_size, self._num_layers, self._embedding_tf)
        self._pooling_tf = self._create_maxpooling_layer(self._convolution_tf)

        return self._pooling_tf

    def summary(self):
        print('embedding:', str(self._embedding_tf.shape))
        print('conv:', str(self._convolution_tf.shape))
        print('pool:', str(self._pooling_tf.shape))

    def _create_embedding_layer(self, embedding, input_x):
        embedding = tf.Variable(initial_value=embedding)

        embedded_chars = tf.nn.embedding_lookup(
            embedding, tf.cast(input_x, 'int32'))

        return embedded_chars

    def _create_convolutional_layers(self, conv_size, num_layers, embedding):
        filter_height = conv_size
        filter_width = embedding.shape[2].value

        filter_shape = [filter_height, filter_width, filter_width]

        W = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=filter_shape,
                stddev=0.1))
        b = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=[filter_width]))

        z = embedding
        for _ in range(num_layers):
            conv = tf.nn.conv1d(
                value=z,
                filters=W,
                stride=1,
                padding='SAME')
            bias = tf.nn.bias_add(conv, b)
            c = tf.nn.relu(bias)

            d = tf.nn.dropout(c, 0.75)
            # Add BatchNormalization or LocalResponseNormalization
            e = tf.expand_dims(d, 1)

            z = tf.nn.local_response_normalization(
                e,
                depth_radius=5,
                bias=1,
                alpha=0.001,
                beta=0.75
            )
            z = tf.squeeze(z, 1)
        # endfor
        return z

    def _create_maxpooling_layer(self, convolution):
        conv_size = convolution.shape[1].value
        embedding_size = convolution.shape[2].value

        convolution = tf.expand_dims(convolution, -1)
        pooled = tf.nn.max_pool(
            value=convolution,
            ksize=[1, conv_size, 1, 1],
            strides=[1, 1, 1, 1],
            padding='VALID')

        flat = tf.reshape(pooled, [-1, embedding_size])
        return flat

In [8]:
word_vector = embedding.astype('float32')
model = ShinContextualModel(embedding=word_vector)

classifier = Classifier(
    model=model,
    input_length=seq_length,
    output_length=num_classes)

classifier.compile(batch_size=32)
classifier.summary()

input: (?, 41)
embedding: (?, 41, 100)
conv: (?, 41, 100)
pool: (?, 100)
output: (?, 10)


In [9]:
# "table": [37, 3]
kf = KFold(n_splits=10)
train_data = dataset_split([37, 3])
train_split_data(train_data, "table")

table
37 14 93 41
Fold:  1
Epoch: 1, Train Loss:  1.846, Train Acc: 37.31%, Val Loss:  1.553, Val Acc: 51.03%, Time: 8.40s *
Epoch: 2, Train Loss: 0.9289, Train Acc: 71.14%, Val Loss:  1.193, Val Acc: 64.30%, Time: 3.67s *
Epoch: 3, Train Loss: 0.5918, Train Acc: 81.79%, Val Loss:  1.251, Val Acc: 67.72%, Time: 3.63s *
Epoch: 4, Train Loss: 0.4078, Train Acc: 87.47%, Val Loss:  1.221, Val Acc: 71.68%, Time: 3.69s *
Epoch: 5, Train Loss: 0.2769, Train Acc: 91.69%, Val Loss:  1.482, Val Acc: 68.54%, Time: 3.65s 
Epoch: 6, Train Loss: 0.2071, Train Acc: 93.79%, Val Loss:  1.485, Val Acc: 71.00%, Time: 3.63s 
Epoch: 7, Train Loss: 0.1535, Train Acc: 95.36%, Val Loss:  1.415, Val Acc: 69.08%, Time: 3.63s 
Epoch: 8, Train Loss: 0.1257, Train Acc: 96.12%, Val Loss:  1.456, Val Acc: 71.27%, Time: 3.64s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.57     

Epoch: 9, Train Loss: 0.1006, Train Acc: 97.00%, Val Loss:   1.84, Val Acc: 68.09%, Time: 3.30s 
Epoch: 10, Train Loss: 0.07761, Train Acc: 97.56%, Val Loss:   2.17, Val Acc: 67.77%, Time: 3.30s 
Epoch: 11, Train Loss: 0.06432, Train Acc: 98.02%, Val Loss:  2.313, Val Acc: 68.46%, Time: 3.30s 
Epoch: 12, Train Loss:  0.057, Train Acc: 98.30%, Val Loss:  2.227, Val Acc: 68.89%, Time: 3.30s *
Epoch: 13, Train Loss: 0.04959, Train Acc: 98.41%, Val Loss:  2.469, Val Acc: 67.02%, Time: 3.30s 
Epoch: 14, Train Loss: 0.04871, Train Acc: 98.40%, Val Loss:  2.312, Val Acc: 68.89%, Time: 3.30s *
Epoch: 15, Train Loss: 0.03479, Train Acc: 98.91%, Val Loss:  2.631, Val Acc: 68.35%, Time: 3.31s 
Epoch: 16, Train Loss: 0.03314, Train Acc: 98.93%, Val Loss:  2.622, Val Acc: 70.01%, Time: 3.30s *
Epoch: 17, Train Loss: 0.03666, Train Acc: 98.77%, Val Loss:  2.792, Val Acc: 68.09%, Time: 3.29s 
Epoch: 18, Train Loss: 0.03192, Train Acc: 98.93%, Val Loss:  2.785, Val Acc: 68.73%, Time: 3.31s 
Epoch: 19,

Epoch: 6, Train Loss: 0.1848, Train Acc: 94.49%, Val Loss:  2.175, Val Acc: 57.17%, Time: 3.45s *
Epoch: 7, Train Loss: 0.1365, Train Acc: 95.88%, Val Loss:  2.532, Val Acc: 55.72%, Time: 3.44s 
Epoch: 8, Train Loss: 0.1114, Train Acc: 96.69%, Val Loss:  2.745, Val Acc: 55.79%, Time: 3.45s 
Epoch: 9, Train Loss: 0.09052, Train Acc: 97.28%, Val Loss:  2.739, Val Acc: 54.70%, Time: 3.43s 
Epoch: 10, Train Loss: 0.07212, Train Acc: 97.68%, Val Loss:  2.889, Val Acc: 56.01%, Time: 3.45s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.65      0.14      0.23       146
                   Filter       0.38      0.38      0.38       117
    Compute Derived Value       0.42      0.49      0.45       152
            Find Extremum       0.55      0.85      0.67       176
                     Sort       0.56      0.85      0.67       123
          Determine Rang

[0.7099863201094391,
 0.6590662323561346,
 0.6454280155642024,
 0.6803409696323921,
 0.6458724202626641,
 0.6435087719298246,
 0.562272396212673,
 0.6706849315068493,
 0.6619915848527349,
 0.6435845213849287]

In [ ]:
kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data = dataset_split(info)
    test_acc_split.append(train_split_data(train_data, split_type))

random
14035 41 41
Fold:  1
Epoch: 1, Train Loss:  1.755, Train Acc: 40.96%, Val Loss:  1.042, Val Acc: 67.74%, Time: 6.72s *
Epoch: 2, Train Loss: 0.7628, Train Acc: 76.26%, Val Loss:  0.615, Val Acc: 82.76%, Time: 3.45s *
Epoch: 3, Train Loss: 0.4345, Train Acc: 86.82%, Val Loss: 0.4223, Val Acc: 88.46%, Time: 3.41s *
Epoch: 4, Train Loss: 0.2788, Train Acc: 91.43%, Val Loss: 0.3144, Val Acc: 91.24%, Time: 3.40s *
Epoch: 5, Train Loss: 0.1935, Train Acc: 94.38%, Val Loss: 0.3032, Val Acc: 92.38%, Time: 3.44s *
Epoch: 6, Train Loss:  0.143, Train Acc: 95.81%, Val Loss: 0.2861, Val Acc: 93.16%, Time: 3.46s *
Epoch: 7, Train Loss: 0.1087, Train Acc: 96.81%, Val Loss: 0.2478, Val Acc: 93.80%, Time: 3.47s *
Epoch: 8, Train Loss: 0.07921, Train Acc: 97.54%, Val Loss: 0.2296, Val Acc: 94.52%, Time: 3.43s *
Epoch: 9, Train Loss: 0.07206, Train Acc: 97.75%, Val Loss:  0.241, Val Acc: 94.09%, Time: 3.42s 
Epoch: 10, Train Loss: 0.05144, Train Acc: 98.46%, Val Loss:  0.247, Val Acc: 94.37%, Tim

Epoch: 1, Train Loss:  1.822, Train Acc: 37.83%, Val Loss:  1.072, Val Acc: 67.02%, Time: 3.54s *
Epoch: 2, Train Loss: 0.7966, Train Acc: 75.03%, Val Loss: 0.6063, Val Acc: 81.84%, Time: 3.41s *
Epoch: 3, Train Loss: 0.4533, Train Acc: 86.27%, Val Loss: 0.4454, Val Acc: 86.68%, Time: 3.44s *
Epoch: 4, Train Loss: 0.2934, Train Acc: 91.47%, Val Loss: 0.3219, Val Acc: 90.03%, Time: 3.44s *
Epoch: 5, Train Loss: 0.2037, Train Acc: 93.79%, Val Loss: 0.2876, Val Acc: 91.31%, Time: 3.41s *
Epoch: 6, Train Loss: 0.1504, Train Acc: 95.58%, Val Loss: 0.2628, Val Acc: 92.74%, Time: 3.55s *
Epoch: 7, Train Loss: 0.1105, Train Acc: 96.66%, Val Loss: 0.2568, Val Acc: 92.17%, Time: 3.42s 
Epoch: 8, Train Loss: 0.08546, Train Acc: 97.35%, Val Loss: 0.2317, Val Acc: 93.23%, Time: 3.43s *
Epoch: 9, Train Loss: 0.07335, Train Acc: 97.71%, Val Loss: 0.2548, Val Acc: 92.81%, Time: 3.42s 
Epoch: 10, Train Loss: 0.05916, Train Acc: 98.20%, Val Loss: 0.2464, Val Acc: 93.52%, Time: 3.43s *
Epoch: 11, Train L

Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.98      0.98      0.98       120
                   Filter       0.93      0.95      0.94       150
    Compute Derived Value       0.97      0.90      0.93       163
            Find Extremum       0.99      0.93      0.96       167
                     Sort       0.91      0.92      0.91       114
          Determine Range       0.84      0.96      0.90       135
Characterize Distribution       0.99      0.84      0.91       137
           Find Anomalies       0.97      0.91      0.94       134
                  Cluster       0.86      0.97      0.91       134
                Correlate       0.91      0.97      0.94       149

                micro avg       0.93      0.93      0.93      1403
                macro avg       0.94      0.93      0.93      1403
             weighted avg       0.94      0.93      0.93      1403

Confusion Matrix...
[[118

Epoch: 3, Train Loss: 0.4541, Train Acc: 86.10%, Val Loss: 0.4013, Val Acc: 87.74%, Time: 3.42s *
Epoch: 4, Train Loss: 0.2953, Train Acc: 91.14%, Val Loss: 0.3006, Val Acc: 90.59%, Time: 3.42s *
Epoch: 5, Train Loss: 0.2079, Train Acc: 93.89%, Val Loss: 0.2388, Val Acc: 92.44%, Time: 3.46s *
Epoch: 6, Train Loss:  0.153, Train Acc: 95.46%, Val Loss: 0.2119, Val Acc: 93.66%, Time: 3.44s *
Epoch: 7, Train Loss: 0.1157, Train Acc: 96.65%, Val Loss: 0.2041, Val Acc: 93.87%, Time: 3.42s *
Epoch: 8, Train Loss: 0.09771, Train Acc: 97.06%, Val Loss: 0.1859, Val Acc: 94.65%, Time: 3.41s *
Epoch: 9, Train Loss: 0.07629, Train Acc: 97.64%, Val Loss: 0.1884, Val Acc: 94.58%, Time: 3.42s 
Epoch: 10, Train Loss: 0.06472, Train Acc: 98.06%, Val Loss: 0.1971, Val Acc: 94.58%, Time: 3.42s 
Epoch: 11, Train Loss: 0.05245, Train Acc: 98.39%, Val Loss:  0.234, Val Acc: 94.58%, Time: 3.42s 
Epoch: 12, Train Loss: 0.05339, Train Acc: 98.23%, Val Loss: 0.2152, Val Acc: 94.08%, Time: 3.43s 
No optimization 

Epoch: 1, Train Loss:  1.773, Train Acc: 39.19%, Val Loss:  1.475, Val Acc: 52.12%, Time: 3.63s *
Epoch: 2, Train Loss: 0.8665, Train Acc: 72.57%, Val Loss:  1.347, Val Acc: 57.43%, Time: 3.51s *
Epoch: 3, Train Loss:  0.537, Train Acc: 83.29%, Val Loss:  1.435, Val Acc: 58.96%, Time: 3.50s *
Epoch: 4, Train Loss: 0.3554, Train Acc: 89.45%, Val Loss:  1.568, Val Acc: 58.42%, Time: 3.49s 
Epoch: 5, Train Loss: 0.2409, Train Acc: 92.94%, Val Loss:  1.659, Val Acc: 59.59%, Time: 3.50s *
Epoch: 6, Train Loss: 0.1743, Train Acc: 95.02%, Val Loss:  1.799, Val Acc: 58.87%, Time: 3.50s 
Epoch: 7, Train Loss: 0.1356, Train Acc: 96.13%, Val Loss:  2.013, Val Acc: 57.34%, Time: 3.51s 
Epoch: 8, Train Loss: 0.1106, Train Acc: 96.70%, Val Loss:  1.936, Val Acc: 60.76%, Time: 3.50s *
Epoch: 9, Train Loss: 0.08582, Train Acc: 97.49%, Val Loss:  2.268, Val Acc: 58.15%, Time: 3.50s 
Epoch: 10, Train Loss: 0.06814, Train Acc: 97.92%, Val Loss:  2.084, Val Acc: 59.50%, Time: 3.50s 
Epoch: 11, Train Loss:

...

Epoch: 5, Train Loss: 0.2491, Train Acc: 92.56%, Val Loss:  1.775, Val Acc: 59.53%, Time: 3.39s 
Epoch: 6, Train Loss: 0.1835, Train Acc: 94.41%, Val Loss:  1.922, Val Acc: 58.39%, Time: 3.39s 
Epoch: 7, Train Loss: 0.1347, Train Acc: 95.95%, Val Loss:  2.111, Val Acc: 57.67%, Time: 3.38s 
Epoch: 8, Train Loss: 0.1034, Train Acc: 96.74%, Val Loss:  2.326, Val Acc: 56.83%, Time: 3.36s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.47      0.58      0.52       112
                   Filter       0.53      0.41      0.46       127
    Compute Derived Value       0.48      0.66      0.55       208
            Find Extremum       0.78      0.47      0.58       329
                     Sort       0.94      0.68      0.79       120
          Determine Range       0.27      0.69      0.38       118
Characterize Distribution       0.53      0.73      0.62  

Epoch: 14, Train Loss: 0.08164, Train Acc: 97.79%, Val Loss:   1.78, Val Acc: 69.12%, Time: 3.43s 
Epoch: 15, Train Loss: 0.0593, Train Acc: 98.05%, Val Loss:  1.775, Val Acc: 69.19%, Time: 3.43s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.42      0.61      0.50        67
                   Filter       0.74      0.63      0.68       166
    Compute Derived Value       0.60      0.68      0.64       203
            Find Extremum       0.68      0.67      0.67       109
                     Sort       0.70      0.72      0.71       151
          Determine Range       0.77      0.51      0.62       215
Characterize Distribution       0.65      0.69      0.67        72
           Find Anomalies       0.74      0.86      0.79       137
                  Cluster       0.75      0.78      0.76       194
                Correlate       0.82      0.82  

Epoch: 1, Train Loss:  2.297, Train Acc: 20.45%, Val Loss:  1.882, Val Acc: 36.96%, Time: 3.55s *
Epoch: 2, Train Loss:  1.452, Train Acc: 51.73%, Val Loss:  1.488, Val Acc: 52.17%, Time: 3.43s *
Epoch: 3, Train Loss: 0.9808, Train Acc: 68.79%, Val Loss:  1.411, Val Acc: 58.41%, Time: 3.43s *
Epoch: 4, Train Loss: 0.7294, Train Acc: 77.34%, Val Loss:  1.324, Val Acc: 60.91%, Time: 3.43s *
Epoch: 5, Train Loss: 0.5415, Train Acc: 83.60%, Val Loss:  1.297, Val Acc: 63.78%, Time: 3.42s *
Epoch: 6, Train Loss: 0.4112, Train Acc: 87.87%, Val Loss:  1.325, Val Acc: 65.47%, Time: 3.42s *
Epoch: 7, Train Loss:  0.314, Train Acc: 90.75%, Val Loss:  1.403, Val Acc: 64.22%, Time: 3.43s 
Epoch: 8, Train Loss: 0.2409, Train Acc: 93.08%, Val Loss:  1.492, Val Acc: 64.73%, Time: 3.44s 
Epoch: 9, Train Loss: 0.1851, Train Acc: 94.63%, Val Loss:  1.507, Val Acc: 65.61%, Time: 3.45s *
Epoch: 10, Train Loss: 0.1476, Train Acc: 95.84%, Val Loss:  1.695, Val Acc: 63.48%, Time: 3.43s 
Epoch: 11, Train Loss:

Epoch: 1, Train Loss:  2.323, Train Acc: 18.52%, Val Loss:  1.828, Val Acc: 36.07%, Time: 3.55s *
Epoch: 2, Train Loss:  1.514, Train Acc: 48.91%, Val Loss:  1.427, Val Acc: 52.64%, Time: 3.42s *
Epoch: 3, Train Loss:  1.039, Train Acc: 67.26%, Val Loss:  1.272, Val Acc: 59.50%, Time: 3.40s *
Epoch: 4, Train Loss: 0.7663, Train Acc: 76.29%, Val Loss:  1.221, Val Acc: 63.21%, Time: 3.41s *
Epoch: 5, Train Loss: 0.5693, Train Acc: 83.35%, Val Loss:  1.164, Val Acc: 64.14%, Time: 3.42s *
Epoch: 6, Train Loss: 0.4439, Train Acc: 87.01%, Val Loss:  1.224, Val Acc: 65.71%, Time: 3.41s *
Epoch: 7, Train Loss: 0.3452, Train Acc: 90.03%, Val Loss:  1.245, Val Acc: 65.50%, Time: 3.41s 
Epoch: 8, Train Loss: 0.2724, Train Acc: 92.15%, Val Loss:   1.29, Val Acc: 68.64%, Time: 3.43s *
Epoch: 9, Train Loss: 0.2108, Train Acc: 94.01%, Val Loss:  1.362, Val Acc: 67.43%, Time: 3.42s 
Epoch: 10, Train Loss: 0.1658, Train Acc: 95.21%, Val Loss:  1.444, Val Acc: 66.93%, Time: 3.42s 
Epoch: 11, Train Loss:

Epoch: 1, Train Loss:  2.388, Train Acc: 17.60%, Val Loss:  1.889, Val Acc: 30.50%, Time: 3.57s *
Epoch: 2, Train Loss:  1.584, Train Acc: 46.95%, Val Loss:  1.261, Val Acc: 59.33%, Time: 3.44s *
Epoch: 3, Train Loss:  1.075, Train Acc: 65.48%, Val Loss:  1.032, Val Acc: 67.04%, Time: 3.45s *
Epoch: 4, Train Loss: 0.7811, Train Acc: 75.98%, Val Loss:  0.977, Val Acc: 71.18%, Time: 3.43s *
Epoch: 5, Train Loss: 0.5913, Train Acc: 82.18%, Val Loss:   0.96, Val Acc: 72.58%, Time: 3.45s *
Epoch: 6, Train Loss: 0.4502, Train Acc: 86.92%, Val Loss:  1.004, Val Acc: 72.93%, Time: 3.44s *
Epoch: 7, Train Loss: 0.3433, Train Acc: 89.92%, Val Loss:  1.005, Val Acc: 74.61%, Time: 3.42s *
Epoch: 8, Train Loss: 0.2653, Train Acc: 92.33%, Val Loss:  1.097, Val Acc: 73.28%, Time: 3.47s 
Epoch: 9, Train Loss: 0.2014, Train Acc: 94.41%, Val Loss:   1.15, Val Acc: 73.07%, Time: 3.45s 
Epoch: 10, Train Loss: 0.1658, Train Acc: 95.40%, Val Loss:  1.199, Val Acc: 73.28%, Time: 3.45s 
